# Practical part Deep learning exam. Student: Ivan Al Khayat


*   importing libraries



In [1]:
from google.colab import drive
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from re import sub, findall
from sklearn.preprocessing import OneHotEncoder,LabelBinarizer, MinMaxScaler
from datetime import datetime
from sklearn.model_selection import train_test_split
import keras
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.layers import Dense, Dropout
from keras import regularizers
import numpy as np


# Loading the dataset and doing Preprocessing [POINT 1: USED FIELDS, TRANSFORMATION AND INPUT OF MY DNN]


In [2]:
df= pd.read_csv("https://github.com/IvanAlKhayat/exam_park_reviews/raw/main/parkReviews.csv", sep= ",", encoding='latin-1')

In [3]:
print(df.shape)
df.dropna(inplace=True) # delete rows with missing fields
print(df.shape)

(42656, 6)
(42656, 6)


In [4]:
df # data exploration

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong
...,...,...,...,...,...,...
42651,1765031,5,missing,United Kingdom,i went to disneyland paris in july 03 and thou...,Disneyland_Paris
42652,1659553,5,missing,Canada,2 adults and 1 child of 11 visited Disneyland ...,Disneyland_Paris
42653,1645894,5,missing,South Africa,My eleven year old daughter and myself went to...,Disneyland_Paris
42654,1618637,4,missing,United States,"This hotel, part of the Disneyland Paris compl...",Disneyland_Paris


In [5]:
df= df[df['Year_Month'] != "missing"] # deleting rows with Year_Month field = 'missing'
print(df.shape)

(40043, 6)


In [6]:
np.unique(df["Year_Month"])

array(['2010-10', '2010-11', '2010-12', '2010-3', '2010-4', '2010-5',
       '2010-6', '2010-7', '2010-8', '2010-9', '2011-1', '2011-10',
       '2011-11', '2011-12', '2011-2', '2011-3', '2011-4', '2011-5',
       '2011-6', '2011-7', '2011-8', '2011-9', '2012-1', '2012-10',
       '2012-11', '2012-12', '2012-2', '2012-3', '2012-4', '2012-5',
       '2012-6', '2012-7', '2012-8', '2012-9', '2013-1', '2013-10',
       '2013-11', '2013-12', '2013-2', '2013-3', '2013-4', '2013-5',
       '2013-6', '2013-7', '2013-8', '2013-9', '2014-1', '2014-10',
       '2014-11', '2014-12', '2014-2', '2014-3', '2014-4', '2014-5',
       '2014-6', '2014-7', '2014-8', '2014-9', '2015-1', '2015-10',
       '2015-11', '2015-12', '2015-2', '2015-3', '2015-4', '2015-5',
       '2015-6', '2015-7', '2015-8', '2015-9', '2016-1', '2016-10',
       '2016-11', '2016-12', '2016-2', '2016-3', '2016-4', '2016-5',
       '2016-6', '2016-7', '2016-8', '2016-9', '2017-1', '2017-10',
       '2017-11', '2017-12', '2017-2', '

# Applying text preprocessing for textual information

ATTENTION: CHANGE WITH RESPECT TO WHAT WRITTEN ON PAPER: IN THE PAPER I WROTE THAT 50 WOULD BE THE REVIEW LENGTH BEING CUT,  BUT FOR THE MODEL TO BE MORE POWERFUL I CONSIDER 130 WHICH IS THE MEAN NUM OF WORDS, SO TO NOT LOSE TOO MUCH INFORMATION

In [7]:
#common stopwords in english
stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

def purify_text(s):

  s=s.lower()#PUTTING EVERITHING TO LOWERCASE
  s = sub(r"<.*?>", " ", s) # Replace HTML tags with spaces
  # Replace all punctuation with spaces
  s = sub(r"[^a-zA-Z0-9')\s]", " ", s)
  s = sub(r'\d+', '', s) #remove numbers

  s=s.split()[:130] # CUTTING EACH REVIEW TO HAVE AT MOST 130 words, COUNTRARY TO 50, THAT I WROTE ON PAPER
  s = [word for word in s if word not in stopwords]

  s=' '.join(s) #tokenization based on white-spaces
  return s



In [8]:
# Limit the 'Review_Text' field to have at most 130 words and do purification
df['Review_Text'] = df['Review_Text'].apply(lambda x: purify_text(x))

/tmp/ipython-input-1984253040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Review_Text'] = df['Review_Text'].apply(lambda x: purify_text(x))


In [9]:
df.head(2)

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,ever disneyland anywhere find disneyland hong ...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,since last time visit hk disneyland yet time s...,Disneyland_HongKong


# Creating the bag-of-words representations
**the aim is to get each review as a vector whose entries are computed via td-idf technique**

In [10]:
#max_features is to 200 most important features based on TF-IDF scores
#This can be useful to limit the dimensionality of the feature matrix
coun_vect = TfidfVectorizer(max_features=200)


In [11]:
count_array = coun_vect.fit_transform(df["Review_Text"]).toarray()
count_array.shape


(40043, 200)

here below there are the words considered in the vector representation for each review

In [12]:
coun_vect.get_feature_names_out()

array(['able', 'adults', 'adventure', 'also', 'although', 'always',
       'amazing', 'around', 'attraction', 'attractions', 'back', 'bad',
       'best', 'better', 'big', 'bit', 'busy', 'california', 'can',
       'castle', 'characters', 'child', 'children', 'christmas', 'clean',
       'closed', 'come', 'could', 'crowded', 'crowds', 'daughter', 'day',
       'days', 'definitely', 'different', 'disappointed', 'disney',
       'disneyland', 'early', 'easy', 'end', 'enjoy', 'enjoyed', 'enough',
       'especially', 'even', 'ever', 'every', 'everyone', 'everything',
       'expect', 'expected', 'expensive', 'experience', 'family',
       'fantastic', 'fast', 'feel', 'find', 'fireworks', 'first',
       'florida', 'food', 'found', 'friendly', 'full', 'fun', 'get',
       'getting', 'go', 'going', 'good', 'got', 'great', 'half', 'happy',
       'hk', 'hong', 'hotel', 'hour', 'hours', 'however', 'kids', 'know',
       'kong', 'land', 'last', 'least', 'less', 'like', 'line', 'lines',
       

**here below, working_df will be the preprocessed feature matrix where each row will be the input of the MLP**

In [13]:
working_df= pd.DataFrame(count_array,dtype='float32') # in working_df the columns of reviews, branch, reviewer_location, year_month preprocessed will be added

**dealing with "branch" feature: one-hot encoding**

In [14]:
ohe= OneHotEncoder()
branches=ohe.fit_transform(df[["Branch"]]).toarray()
print(sum(np.isnan(branches)))#sanity check
branches = pd.DataFrame(branches, columns=ohe.categories_[0], dtype=bool)
print(branches.shape)

[0 0 0]
(40043, 3)


In [15]:
print(df["Branch"].value_counts())
working_df["Disneyland_HongKong"]=branches["Disneyland_HongKong"]
working_df["Disneyland_California"]=branches["Disneyland_California"]
working_df["Disneyland_Paris"]=branches["Disneyland_Paris"]

Branch
Disneyland_California    18202
Disneyland_Paris         12694
Disneyland_HongKong       9147
Name: count, dtype: int64



**dealing with "reviewer_location" feature: one-hot encoding**



In [16]:
ohe = OneHotEncoder()
rev_locs=ohe.fit_transform(df[["Reviewer_Location"]])

# Convert the one-hot encoded data to a dense array
rev_locs = rev_locs.toarray()
print("how many rows have some field nan",sum(np.isnan(rev_locs)))#sanity check

# Create a DataFrame from the one-hot encoded array
rev_locs = pd.DataFrame(rev_locs, columns=ohe.categories_[0], dtype=bool)
print(rev_locs.shape)


how many rows have some field nan [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(40043, 162)


In [17]:
rev_locs.shape
rev_locs.dropna()
rev_locs.shape

(40043, 162)

In [18]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
for col in rev_locs.columns:
  working_df[col] = rev_locs[col].values


**dealing with Year_month feature ATTENTION: CHANGE WRT WRITTEN ON PAPER: INSTEAD OF CONSIDERING TIME_IN_MILLIS, I CONSIDER THE NUMBER OF MONTHS PASSED SINCE 1 JAN 1970, otherwise numbers would be too large**

In [19]:
def from_string_to_int(x):
    try:
      date_string = x

# Convert date string to datetime object
      date = datetime.strptime(date_string, "%Y-%m")

# Calculate the number of months passed since 1970
      months_passed = (date.year - 1970) * 12 + (date.month - 1)
      return months_passed

    except ValueError:
      return 0


In [20]:
df["Year_Month"]= df['Year_Month'].apply(lambda x: from_string_to_int(x))

working_df["Year_Month"]=df["Year_Month"]

#replacing nan values with zero
working_df["Year_Month"]= working_df['Year_Month'].replace(to_replace= np.nan,value=0)
print(np.unique(working_df["Year_Month"]))



[  0. 482. 483. 484. 485. 486. 487. 488. 489. 490. 491. 492. 493. 494.
 495. 496. 497. 498. 499. 500. 501. 502. 503. 504. 505. 506. 507. 508.
 509. 510. 511. 512. 513. 514. 515. 516. 517. 518. 519. 520. 521. 522.
 523. 524. 525. 526. 527. 528. 529. 530. 531. 532. 533. 534. 535. 536.
 537. 538. 539. 540. 541. 542. 543. 544. 545. 546. 547. 548. 549. 550.
 551. 552. 553. 554. 555. 556. 557. 558. 559. 560. 561. 562. 563. 564.
 565. 566. 567. 568. 569. 570. 571. 572. 573. 574. 575. 576. 577. 578.
 579. 580. 581. 582. 583. 584. 585. 586. 587. 588. 589. 590. 591. 592.]


/tmp/ipython-input-2372967702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year_Month"]= df['Year_Month'].apply(lambda x: from_string_to_int(x))


In [21]:
del count_array #saves memory

In [22]:
working_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40043 entries, 0 to 40042
Columns: 366 entries, 0 to Year_Month
dtypes: bool(165), float32(200), float64(1)
memory usage: 37.2 MB


In [23]:
working_df.shape

(40043, 366)

In [24]:
del coun_vect #saves memory

In [25]:
working_df.info()
print(working_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40043 entries, 0 to 40042
Columns: 366 entries, 0 to Year_Month
dtypes: bool(165), float32(200), float64(1)
memory usage: 37.2 MB
(40043, 366)


# Performing scaling
the field Year_month contains huge numbers, we scale them to range from 0 to 1

In [26]:
#scaling year_month field
print(working_df["Year_Month"][3])#example of before
print(working_df.Year_Month.value_counts)

scaler= MinMaxScaler()
working_df["Year_Month"]= scaler.fit_transform(working_df[["Year_Month"]])
print(working_df["Year_Month"][3])#example of after



591.0
<bound method IndexOpsMixin.value_counts of 0        591.0
1        592.0
2        591.0
3        591.0
4        591.0
         ...  
40038    518.0
40039    518.0
40040    518.0
40041    518.0
40042    511.0
Name: Year_Month, Length: 40043, dtype: float64>
0.9983108108108109


need to rescale the Ratings from 0 to 4, so to have 5 units in the output layer, otherwise i need 6 units but that means including zero in the predictions, which is never present

In [27]:
y= df["Rating"]
y= y.apply(lambda x: x-1)
np.unique(y)

array([0, 1, 2, 3, 4])

# Checking label balancing

In [28]:
print(len(y))
y.value_counts() # labels are slightly unbalanced, need to use weighted f1 score

40043


,count
Rating,
4,21908
3,10086
2,4782
1,1929
0,1338


# Train and Test dataset splitting

In [29]:
X_train, X_test, y_train, y_test = train_test_split(working_df, y, test_size=0.33, random_state=42)

In [30]:
del df

In [31]:
X_train[0]

,0
23943,0.0
16584,0.0
21968,0.0
4513,0.0
4596,0.0
...,...
6265,0.0
11284,0.0
38158,0.0
860,0.0


In [32]:
X_train = np.asarray(X_train).astype(np.float64)
y_train = np.asarray(y_train).astype(np.float64)

X_test = np.asarray(X_train).astype(np.float64)
y_test = np.asarray(y_train).astype(np.float64)


# Performing model selection: it's been performed by creating a baseline model and tuning 1 hyperparam at a time, and selecting the best performing model to undergo further tuning on the remaining hyperparams, in a greedy way

# [POINT 6: THE DATASET IS UNBALANCED. I'M EVALUATING GENERALIZATION CAPABILITIES ON THE TEST SET, BASED ON THE F1-WEIGHTED SCORE, high score means high generalization capability]

In [33]:
from sklearn.metrics import f1_score

def get_f1_score(model):#function used to evaluate models
  pred = model.predict(X_test)

  cat_pred=np.argmax(pred, axis=1)+1 # +1 because I'm returning to the original rating scale

  print("weighted f1 score: ",f1_score(y_test+1, cat_pred, average='weighted'))


In [34]:
np.unique(y_train)

array([0., 1., 2., 3., 4.])

# [POINT 2: OUTPUT IS A DENSE LAYER WITH 5 NEURONS AND SOFTMAX ACTIVATION FUNCTION]

In [35]:
def get_model(#standard params for baseline model
    neurons=[64, 64, 64],
    activations=['relu','sigmoid','tanh'],
    act_regularizers=[None, None, None],
    weight_regularizers=[None, None, None],
    dropout=[0,0,0]
):

  model= keras.Sequential(
    [
        Dense(input_shape=(X_train.shape[1],),units=neurons[0], activation=activations[0], kernel_regularizer= weight_regularizers[0],
              kernel_initializer= keras.initializers.HeUniform() if activations[0]=='relu' else keras.initializers.GlorotNormal() ,
               activity_regularizer= act_regularizers[0]),
        keras.layers.Dropout(dropout[0],),

        Dense(input_shape=(X_train.shape[1],),units=neurons[1], activation=activations[1], kernel_regularizer= weight_regularizers[1],
              kernel_initializer= keras.initializers.HeUniform() if activations[1]=='relu' else keras.initializers.GlorotNormal() ,
              activity_regularizer= act_regularizers[1]),
        keras.layers.Dropout(dropout[1],),

        Dense(input_shape=(X_train.shape[1],),units=neurons[2], activation=activations[2], kernel_regularizer= weight_regularizers[2],
              kernel_initializer= keras.initializers.HeUniform() if activations[2]=='relu' else keras.initializers.GlorotNormal() ,
              activity_regularizer= act_regularizers[2]),
        keras.layers.Dropout(dropout[2],),

        Dense(units=5, activation="softmax")

    ])
  print(model.summary())
  return model


# [POINT 3: WHILE TRYING SOME SENSIBLE CONFIGURATIONS, YOU CAN FIND THE ACTIVATION FUNCTIONS THAT I USED THAT ARE RELU, SIGMOID AND TANH]

# [POINT 5: HERE BELOW THERE ARE THE HYPERPARAMETERS ON WHICH I PERFORMED MANUAL MODEL SELECTION]

**trying the baseline model**

In [40]:
model= get_model(); #uses relu, sigmoid and tanh

model.compile(optimizer=keras.optimizers.Adam(learning_rate=10**-2, clipvalue=0.5),
                loss=keras.losses.SparseCategoricalCrossentropy(
    from_logits=False,
    ignore_class=None,
    reduction="sum_over_batch_size",
    name="sparse_categorical_crossentropy",
),
                metrics=['accuracy'])

epochs = 150
batch_size = 1000
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,validation_split=0.2,
          shuffle=True,
          callbacks= [keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
get_f1_score(model)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 64)             │        23,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,133 (125.52 KB)

 Trainable params: 32,133 (125.52 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 103ms/step - accuracy: 0.4627 - loss: 1.3681 - val_accuracy: 0.5604 - val_loss: 1.1319
Epoch 2/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5674 - loss: 1.0834 - val_accuracy: 0.5732 - val_loss: 1.0422
Epoch 3/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5824 - loss: 0.9994 - val_accuracy: 0.5785 - val_loss: 1.0085
Epoch 4/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5912 - loss: 0.9570 - val_accuracy: 0.5809 - val_loss: 1.0010
Epoch 5/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5928 - loss: 0.9513 - val_accuracy: 0.5878 - val_loss: 0.9960
Epoch 6/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6023 - loss: 0.9376 - val_accuracy: 0.5870 - val_loss: 0.9989
Epoch 7/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6054 - loss: 0.9304 - val_accuracy: 0.5861 - val_loss: 1.0025
Epoch 8/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6180 - loss: 0.9068 - val_accuracy: 0.5

**trying to increase number of neurons for hidden layers**


In [43]:
model= get_model(neurons=[256,1024,512]);

model.compile(optimizer=keras.optimizers.Adam(learning_rate=10**-2, clipvalue=0.5),
                loss=keras.losses.SparseCategoricalCrossentropy(
    from_logits=False,
    ignore_class=None,
    reduction="sum_over_batch_size",
    name="sparse_categorical_crossentropy",
),
                metrics=['accuracy'])

epochs = 150
batch_size = 1000
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,validation_split=0.2,
          shuffle=True,
          callbacks= [keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
get_f1_score(model)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (None, 256)            │        93,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 884,485 (3.37 MB)

 Trainable params: 884,485 (3.37 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - accuracy: 0.3427 - loss: 4.7224 - val_accuracy: 0.1208 - val_loss: 2.3619
Epoch 2/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.3873 - loss: 1.7492 - val_accuracy: 0.5458 - val_loss: 1.2899
Epoch 3/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5538 - loss: 1.2447 - val_accuracy: 0.5458 - val_loss: 1.2026
Epoch 4/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5466 - loss: 1.2017 - val_accuracy: 0.5458 - val_loss: 1.1948
Epoch 5/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5435 - loss: 1.1988 - val_accuracy: 0.5458 - val_loss: 1.1967
Epoch 6/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5515 - loss: 1.2002 - val_accuracy: 0.5458 - val_loss: 1.2145
Epoch 7/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5464 - loss: 1.2052 - val_accuracy: 0.5458 - val_loss: 1.2261
Epoch 8/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5434 - loss: 1.2128 - val_accuracy: 0

**changing activation functions**

In [44]:
model= get_model(    activations=['relu','relu','relu'],
)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=10**-2, clipvalue=0.5),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])

epochs = 150
batch_size = 1000
model.fit(X_train, y_train,batch_size=batch_size,
          epochs=epochs,validation_split=0.2,
          shuffle=True,callbacks= [keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
get_f1_score(model)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_32 (Dense)                │ (None, 64)             │        23,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,133 (125.52 KB)

 Trainable params: 32,133 (125.52 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.4869 - loss: 1.2552 - val_accuracy: 0.5570 - val_loss: 1.0625
Epoch 2/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5701 - loss: 1.0187 - val_accuracy: 0.5758 - val_loss: 1.0136
Epoch 3/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5896 - loss: 0.9574 - val_accuracy: 0.5867 - val_loss: 0.9963
Epoch 4/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6046 - loss: 0.9276 - val_accuracy: 0.5833 - val_loss: 1.0037
Epoch 5/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6188 - loss: 0.8957 - val_accuracy: 0.5844 - val_loss: 1.0197
Epoch 6/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6330 - loss: 0.8724 - val_accuracy: 0.5854 - val_loss: 1.0357
Epoch 7/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6508 - loss: 0.8344 - val_accuracy: 0.5721 - val_loss: 1.0592
Epoch 8/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6621 - loss: 0.8117 - val_accuracy: 0.56

# [POINT 4: TRYING ACTIVITY AND WEIGHT REGULARIZER, DID NOT TRY ANY BATCH NORMALIZATION FOR SIMPLICITY]

**adding activity and weight regularizers**

In [45]:
model= get_model(    activations=['relu','relu','relu'],
                 act_regularizers=['L1', 'L1', 'L2'],
    weight_regularizers=['L1', 'L1', 'L2'],
)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=10**-2, clipvalue=0.5),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])
epochs = 150
batch_size = 1000
model.fit(X_train, y_train,batch_size=batch_size,
          epochs=epochs,validation_split=0.2,
          shuffle=True,callbacks= [keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
get_f1_score(model)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                │ (None, 64)             │        23,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,133 (125.52 KB)

 Trainable params: 32,133 (125.52 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - accuracy: 0.4218 - loss: 39.5828 - val_accuracy: 0.5458 - val_loss: 13.1649
Epoch 2/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5470 - loss: 11.6146 - val_accuracy: 0.5458 - val_loss: 7.7025
Epoch 3/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5498 - loss: 6.8608 - val_accuracy: 0.5458 - val_loss: 4.8358
Epoch 4/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5476 - loss: 4.3928 - val_accuracy: 0.5458 - val_loss: 3.3120
Epoch 5/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5493 - loss: 3.0650 - val_accuracy: 0.5458 - val_loss: 2.4658
Epoch 6/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5431 - loss: 2.3305 - val_accuracy: 0.5458 - val_loss: 1.9781
Epoch 7/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5481 - loss: 1.8953 - val_accuracy: 0.5458 - val_loss: 1.7069
Epoch 8/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5488 - loss: 1.6632 - val_accuracy: 

**trying dropout**

In [54]:
model= get_model(    activations=['relu','relu','relu'],
                 act_regularizers=['L1', 'L1', 'L2'],
    weight_regularizers=['L1', 'L1', 'L2'],
                 dropout=[0.8,0.3,0]
)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=10**-2, clipvalue=0.5),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])

epochs = 150
batch_size = 1000
model.fit(X_train, y_train,batch_size=batch_size,
          epochs=epochs,validation_split=0.2,
          shuffle=True,callbacks= [keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
get_f1_score(model)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_72 (Dense)                │ (None, 64)             │        23,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_54 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_73 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_55 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_74 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_56 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_75 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,133 (125.52 KB)

 Trainable params: 32,133 (125.52 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 139ms/step - accuracy: 0.4519 - loss: 57.4929 - val_accuracy: 0.5458 - val_loss: 16.7077
Epoch 2/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5442 - loss: 15.3029 - val_accuracy: 0.5458 - val_loss: 11.1913
Epoch 3/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5448 - loss: 10.0963 - val_accuracy: 0.5458 - val_loss: 7.2525
Epoch 4/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5481 - loss: 6.5517 - val_accuracy: 0.5458 - val_loss: 4.7615
Epoch 5/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5428 - loss: 4.3374 - val_accuracy: 0.5458 - val_loss: 3.2321
Epoch 6/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5465 - loss: 2.9738 - val_accuracy: 0.5458 - val_loss: 2.3379
Epoch 7/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5413 - loss: 2.2034 - val_accuracy: 0.5458 - val_loss: 1.8680
Epoch 8/150
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5444 - loss: 1.7963 - val_accuracy

**trying tuning the learning rate**

In [56]:
model= get_model(    activations=['relu','relu','relu'],
)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=10**-3, clipvalue=0.5), # 10^-3 vs 10^-2 of prev. models
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])

epochs = 350
batch_size = 1000
model.fit(X_train, y_train,batch_size=batch_size,
          epochs=epochs,validation_split=0.2,
          shuffle=True,callbacks= [keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
get_f1_score(model)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_80 (Dense)                │ (None, 64)             │        23,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_60 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_61 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_62 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_83 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,133 (125.52 KB)

 Trainable params: 32,133 (125.52 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.4594 - loss: 1.4931 - val_accuracy: 0.5458 - val_loss: 1.2426
Epoch 2/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5498 - loss: 1.2044 - val_accuracy: 0.5475 - val_loss: 1.1461
Epoch 3/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5486 - loss: 1.1240 - val_accuracy: 0.5542 - val_loss: 1.0966
Epoch 4/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5666 - loss: 1.0618 - val_accuracy: 0.5626 - val_loss: 1.0584
Epoch 5/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5708 - loss: 1.0266 - val_accuracy: 0.5704 - val_loss: 1.0321
Epoch 6/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5830 - loss: 0.9926 - val_accuracy: 0.5783 - val_loss: 1.0174
Epoch 7/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5925 - loss: 0.9712 - val_accuracy: 0.5768 - val_loss: 1.0093
Epoch 8/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5967 - loss: 0.9565 - val_accuracy: 0.57

**tuning the batch size**

In [57]:
model1= model2= get_model(    activations=['relu','relu','relu'],
)

model1.compile(optimizer=keras.optimizers.Adam(learning_rate=10**-2, clipvalue=0.5),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])
model2.compile(optimizer=keras.optimizers.Adam(learning_rate=10**-2, clipvalue=0.5),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])

epochs = 15
batch_size = 100 #lower batch size
model1.fit(X_train, y_train,batch_size=batch_size,
          epochs=epochs,validation_split=0.2,
          shuffle=True,callbacks= [keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
get_f1_score(model1)
## ---vs---##
epochs = 15
batch_size = 20000 #higher batch size
model2.fit(X_train, y_train,batch_size=batch_size,
          epochs=epochs,validation_split=0.2,
          shuffle=True,callbacks= [keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
get_f1_score(model2)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_84 (Dense)                │ (None, 64)             │        23,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_63 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_85 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_64 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_86 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_65 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_87 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,133 (125.52 KB)

 Trainable params: 32,133 (125.52 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/15
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.5513 - loss: 1.0902 - val_accuracy: 0.5751 - val_loss: 1.0136
Epoch 2/15
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5848 - loss: 0.9671 - val_accuracy: 0.5747 - val_loss: 0.9975
Epoch 3/15
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5984 - loss: 0.9423 - val_accuracy: 0.5816 - val_loss: 1.0216
Epoch 4/15
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6076 - loss: 0.9177 - val_accuracy: 0.5779 - val_loss: 1.0266
Epoch 5/15
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6222 - loss: 0.8808 - val_accuracy: 0.5731 - val_loss: 1.0466
Epoch 6/15
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6359 - loss: 0.8561 - val_accuracy: 0.5727 - val_loss: 1.0828
Epoch 7/15
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6457 - loss: 0.8338 - val_accuracy: 0.5745 - val_loss: 1.1341
Epoch 8/15
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6620 - loss: 0.8054 - val_accura

finally, the model to select will be the one with highest f1-weighted.